## Experimental Research on Tagged ArXiV Data

In [ ]:
import pandas as pd
#import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

NUM_AUTHORS_RECORDED = 6


### Notes:

* Potential Prophet Regressors -- Num. Unique authors, Mean num. publications of authors publishing in a given window (speaks to maturity of field, might be hard to get), pages (fuzzy extract from comments), **total publications to ArXiV in a given discipline** (to mitigate bias by the overall growth of arxiv), percent of publications in a subfield given an author/field.
* Eventually switch from Pandas to PySpark for SQL Querying



In [161]:
math_df = pd.read_csv('/Users/yavuzdev/Desktop/LectureNotes/MAIS202/Project/math_arxiv_data.csv')

math_df['created_date'] = pd.to_datetime(math_df['latest_created_date']).dt.date
math_df['created_month'] = pd.to_datetime(math_df['created_date']).dt.to_period('M')

##math_categories_df = math_df['categories'].str.split(' ', expand=True, n=3)
math_categories_df = math_df['categories'].str.split(' ').explode()

# math_categories_df = math_categories_df.rename(
#     columns= {
#         0: 'cat_1',
#         1: 'cat_2',
#         2: 'cat_3',
#         3: 'cat_4',
#         4: 'cat_5',
#         5: 'cat_6',
#         6: 'cat_7',
#         7: 'cat_8',
#         8: 'cat_9',
#         9: 'cat_10',
#         10: 'cat_11',
#         11: 'cat_12',
#         12: 'cat_13'
#     }
# )

display(math_categories_df)
# display(math_df['created_month'])
# print(type(math_df['created_month'][1]))
#id,authors,title,categories,comments,update_date,latest_created_date

/var/folders/qz/fvkql1ds5dz_hwlp8jszf4180000gp/T/ipykernel_1654/3457970814.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  math_df = pd.read_csv('/Users/yavuzdev/Desktop/LectureNotes/MAIS202/Project/math_arxiv_data.csv')


0          math.CO
0            cs.CG
1          math.CO
2          math.CA
2          math.FA
            ...   
687237    solv-int
687237      hep-th
687237     math-ph
687237     math.MP
687237     nlin.SI
Name: categories, Length: 1359075, dtype: object

In [116]:

march25= pd.Period('2025-03-01', freq='M')
print(march25)
print(type(march25))
#dt.period('2016-08-09')



2025-03
<class 'pandas._libs.tslibs.period.Period'>


In [162]:
math_authors_uncleaned_df = math_df['authors']
math_authors_df = math_df['authors'].str.replace(' and ', ', ').str.split(', ', expand=True, n= NUM_AUTHORS_RECORDED - 1)

math_authors_df = math_authors_df.rename(
    columns={
        0: 'auth_1',
        1: 'auth_2',
        2: 'auth_3',
        3: 'auth_4',
        4: 'auth_5',
        5: 'auth_6',
        6: 'auth_7',
        7: 'auth_8',
        8: 'auth_9',
        9: 'auth_10',
        10: 'auth_11'
    }
)

math_authors_df['num_authors'] = math_df['authors'].str.replace(' and ', ', ').str.split(', ').str.len()

display(math_authors_df)

,auth_1,auth_2,auth_3,auth_4,auth_5,auth_6,num_authors
0,Ileana Streinu,Louis Theran,None,None,None,None,2
1,David Callan,None,None,None,None,None,1
2,Wael Abu-Shammala,Alberto Torchinsky,None,None,None,None,2
3,Sergei Ovchinnikov,None,None,None,None,None,1
4,Clifton Cunningham,Lassina Dembele,None,None,None,None,2
...,...,...,...,...,...,...,...
687233,A.J. Macfarlane,F. Wagner,None,None,None,None,2
687234,Angel Ballesteros,Francisco J. Herranz,None,None,None,None,2
687235,F.Delduc,A.Sorin,None,None,None,None,2
687236,Olaf Lechtenfeld,Alexander Sorin,None,None,None,None,2


In [164]:

math_base_df = math_df[['id', 'title', 'comments', 'created_date', 'created_month']]

tables = [math_base_df, math_authors_df, math_categories_df]

#id,authors,title,categories,comments,update_date,latest_created_date

math_arxiv_df = pd.concat(tables, axis=1)#.sort_values(by='created_date', ascending=True)
#math_arxiv_df['created_date'] = math_arxiv_df['latest_created_date'].str[0:10]

display(math_arxiv_df)


,id,title,comments,created_date,created_month,auth_1,auth_2,auth_3,auth_4,auth_5,auth_6,num_authors,categories
0,704.0002,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,2008-12-13,2008-12,Ileana Streinu,Louis Theran,None,None,None,None,2,math.CO
0,704.0002,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,2008-12-13,2008-12,Ileana Streinu,Louis Theran,None,None,None,None,2,cs.CG
1,704.0004,A determinant of Stirling cycle numbers counts...,11 pages,2007-03-31,2007-03,David Callan,None,None,None,None,None,1,math.CO
2,704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,NaN,2007-04-02,2007-04,Wael Abu-Shammala,Alberto Torchinsky,None,None,None,None,2,math.CA
2,704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,NaN,2007-04-02,2007-04,Wael Abu-Shammala,Alberto Torchinsky,None,None,None,None,2,math.FA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
687237,solv-int/9912015,Vector NLS hierarchy solitons revisited: dress...,"LaTex, 41 pages",1999-12-27,1999-12,Harold Blas,None,None,None,None,None,1,solv-int
687237,solv-int/9912015,Vector NLS hierarchy solitons revisited: dress...,"LaTex, 41 pages",1999-12-27,1999-12,Harold Blas,None,None,None,None,None,1,hep-th
687237,solv-int/9912015,Vector NLS hierarchy solitons revisited: dress...,"LaTex, 41 pages",1999-12-27,1999-12,Harold Blas,None,None,None,None,None,1,math-ph
687237,solv-int/9912015,Vector NLS hierarchy solitons revisited: dress...,"LaTex, 41 pages",1999-12-27,1999-12,Harold Blas,None,None,None,None,None,1,math.MP


## Get Unique Authors for a Given Month

In [ ]:
#E.g. March 2025

def get_unique_authors_by_monthly_window(discipline_df, created_month, category):
    '''
    Extracts the unique authors in a given table of publications.

    Takes:
      - A DataFrame of publications
      - A string in the format 'YYYY-MM.

    Returns:
      - The number of unique authors
      - A list of unique authors.
    '''

    created_month_period = pd.Period(created_month, freq='M')

    # Ensure the 'created_month' column is in Period format
    monthly_discipline_arxiv_df = discipline_df[discipline_df['created_month'] == created_month] #AND category is category
    #display(march25_math_arxiv_df)

    author_cols = [monthly_discipline_arxiv_df[f'auth_{i + 1}'] for i in range(NUM_AUTHORS_RECORDED)]
    monthly_authors = pd.concat(author_cols, axis=0)
    monthly_unique_authors = monthly_authors.drop_duplicates()
    num_unique_authors = monthly_unique_authors.shape[0]

    return num_unique_authors, monthly_unique_authors



# march25_math_arxiv_df = math_arxiv_df[math_arxiv_df['created_date'].str[0:7] == '2025-03']
# #display(march25_math_arxiv_df)

# author_cols = [march25_math_arxiv_df[f'auth_{i + 1}'] for i in range(NUM_AUTHORS_RECORDED)]
# march25_math_arxiv_authors = pd.concat(author_cols, axis=0)
# march25_math_arxiv_unique_authors = march25_math_arxiv_authors.drop_duplicates()
# num_unique_authors = march25_math_arxiv_unique_authors.shape[0]

# display(march25_math_arxiv_unique_authors)

# print(num_unique_authors)


In [166]:
num_unique_authors, monthly_unique_authors = get_unique_authors_by_monthly_window(math_arxiv_df, '1999-12')

print(num_unique_authors)

458


## Aggregated Count Data per Month

In [169]:
#Count number of publications per month
#math_arxiv_agg_df = math_arxiv_df.groupby('created_month').agg(count('id').alias('num_pubs')).sort('created_month')
math_arxiv_agg_df = math_arxiv_df.groupby(['created_month', 'categories'])['created_month'].count()
math_arxiv_agg_df = math_arxiv_agg_df.reset_index(name='num_pubs')


print(math_arxiv_agg_df)

      created_month categories  num_pubs
0           1989-04    math.LO         1
1           1989-10    math.FA         3
2           1989-10    math.MG         3
3           1989-11    math.FA         1
4           1989-11    math.MG         1
...             ...        ...       ...
34454       2025-03    stat.CO        31
34455       2025-03    stat.ME       100
34456       2025-03    stat.ML       223
34457       2025-03    stat.OT         4
34458       2025-03    stat.TH       315

[34459 rows x 3 columns]


In [170]:
author_counts_arr = np.zeros(math_arxiv_agg_df['created_month'].shape[0])


for i, month in enumerate(math_arxiv_agg_df['created_month']):
    num_unique_authors, monthly_unique_authors = get_unique_authors_by_monthly_window(math_arxiv_df, month)
    author_counts_arr[i] = num_unique_authors

math_arxiv_agg_df['num_unique_authors'] = author_counts_arr.astype(int)
math_arxiv_agg_df['mean_auths_per_pub'] =  math_arxiv_agg_df['num_unique_authors'] / math_arxiv_agg_df['num_pubs']

#display(math_arxiv_agg_df[math_arxiv_agg_df['created_month'] >= pd.Period('2020-01')])

In [171]:
display(math_arxiv_agg_df[math_arxiv_agg_df['created_month'] >= pd.Period('2020-01')])

,created_month,categories,num_pubs,num_unique_authors,mean_auths_per_pub
25975,2020-01,astro-ph,1,7121,7121.000000
25976,2020-01,astro-ph.CO,2,7121,3560.500000
25977,2020-01,astro-ph.EP,3,7121,2373.666667
25978,2020-01,astro-ph.IM,2,7121,3560.500000
25979,2020-01,cond-mat.dis-nn,14,7121,508.642857
...,...,...,...,...,...
34454,2025-03,stat.CO,31,14106,455.032258
34455,2025-03,stat.ME,100,14106,141.060000
34456,2025-03,stat.ML,223,14106,63.255605
34457,2025-03,stat.OT,4,14106,3526.500000
